# Cuaderno 6: Convoluciones

En este cuaderno mostraremos algunos conceptos básicos sobre la convolución.

Para comprender la visión humana, es esencial entender las primeras etapas del procesamiento de imágenes en la vía visual. Un problema común al encontrar bordes es que las imágenes no son perfectas; contienen imperfecciones conocidas como ruido. La teoría aborda el problema del ruido desenfocando las imágenes, mediante un proceso llamado convolución. Básicamente, consiste en aplicar un tipo particular de operador a lo largo de la imagen. Este operador tiene un perfil gaussiano porque la teoría computacional especifica que esta es la forma que optimiza la combinación de suavizar el ruido sin afectar demasiado las zonas donde se encuentran los bordes en la imagen convolucionada. El siguiente paso es identificar las regiones de la imagen donde hay cambios abruptos en las intensidades, ya que ahí es donde están los bordes. Esto requiere medir gradientes de intensidad y/o cambios en los gradientes, que se conocen como la primera y segunda derivadas, respectivamente. Se describen varios algoritmos biológicamente plausibles para implementar la teoría. Estos algoritmos implican operadores notablemente similares a los campos receptivos de las células en la retina y la corteza estriada.

Las representaciones cerebrales de los bordes no pueden ser lo único implicado en la visión, ya que somos capaces de describir características de la escena mucho más complejas que los bordes. Aun así, las representaciones de los bordes pueden ser útiles de inmediato para guiar una acción de agarre en torno a un objeto. Además, pueden servir como un primer paso importante en tareas perceptuales más complejas, como el reconocimiento de objetos o la percepción de profundidad.

- Capítulos 3, 5 y 9 de Frisby, J. P. & Stone, J. V. *Seeing*. (The MIT Press, London, 2010).
- Capítulo 7 de Trappenberg, T. P. *Fundamentals of Computational Neuroscience*. (Oxford University Press, Oxford, 2022).

## Configuración

Comenzamos importando las librerías que vamos a usar:

In [ ]:
import math

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.image as img
import ipywidgets as widgets

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

### Funciones utilitarias

In [ ]:
def encontrar_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
    print("Device encontrado:", device)
    return device
    
def train_net(model, loader, optimizer, criterion):
    model.train()
    total_loss, correct, total = 0.0, 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * x.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += x.size(0)
    return total_loss/total, correct/total

@torch.no_grad()
def eval_net(model, loader, criterion):
    model.eval()
    total_loss, correct, total = 0.0, 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss = criterion(logits, y)
        total_loss += loss.item() * x.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += x.size(0)
    return total_loss/total, correct/total

### Funciones de graficado

In [ ]:
def visualizar_convolucion1d(s, f, convolucion):
  fig, (ax1, ax2, ax3) = plt.subplots(1, 3, width_ratios=[2, 1, 2], figsize=(12, 3))

  ax1.set_title("Señal original")
  ax1.plot(s)
  ax1.set_ylim(-0.1, 1.1)
  
  ax2.set_title("Filtro")
  ax2.imshow(f.reshape(1, f.shape[0]), cmap="coolwarm", vmin=-1, vmax=1)
  ax2.set_axis_off()
  
  ax3.set_title("Resultado de la convolución")
  ax3.plot(convolucion)
  ax3.set_ylim(-1, 1)
  
  fig.subplots_adjust(wspace=0.4)
  
  pos1 = ax1.get_position()
  pos2 = ax2.get_position()
  pos3 = ax3.get_position()

  x_plus = (pos1.x1 + pos2.x0) / 2
  x_eq = (pos2.x1 + pos3.x0) / 2

  y_center = (pos1.y0 + pos1.y1) / 2

  fig.text(x_plus, y_center, '+', ha='center', va='center', fontsize=16)
  fig.text(x_eq, y_center, '=', ha='center', va='center', fontsize=16)
  
  plt.show()
    
def visualizar_convolucion2d(s, f, convolucion):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, width_ratios=[2, 1, 2], figsize=(12, 6))

    ax1.set_title("Imagen original")
    ax1.matshow(s, cmap="gray")
    ax1.set_axis_off()
    
    ax2.set_title("Filtro")
    ax2.imshow(f, cmap="coolwarm")
    ax2.set_axis_off()
    
    ax3.set_title("Resultado de la convolución")
    ax3.imshow(convolucion, cmap="gray")
    ax3.set_axis_off()
    
    fig.subplots_adjust(wspace=0.2)
    fig.text(0.420, 0.5, '+', ha='center', va='center', fontsize=16)
    fig.text(0.605, 0.5, '=', ha='center', va='center', fontsize=16)
    
    plt.show()


def visualizar_pesos(conv_layer, in_channel=0, max_plots=64):
  W = conv_layer.weight.detach().cpu()  # (out_ch, in_ch, kH, kW)
  n = min(W.shape[0], max_plots)
  cols = int(math.ceil(math.sqrt(n)))
  rows = int(math.ceil(n / cols))
  
  fig, axes = plt.subplots(rows, cols, figsize=(2*cols, 2*rows))
  axes = axes.flatten()
  
  for i in range(n):
      ker = W[i, in_channel].numpy()
      axes[i].imshow(ker, cmap="bwr")  # bwr: azul negativo, rojo positivo
      axes[i].axis("off")
  for j in range(n, len(axes)):
      axes[j].axis("off")
  plt.suptitle(f"Pesos {conv_layer.__class__.__name__}")
  plt.show()
    

## Exploración

*Imagenette* es un subconjunto reducido del famoso dataset ImageNet, diseñado para fines educativos y de experimentación rápida en visión por computadora. Contiene 10 clases seleccionadas y un número mucho menor de imágenes que el conjunto original, lo que permite entrenar y evaluar modelos en menos tiempo sin perder el realismo de trabajar con imágenes naturales.

Para comenzar, ejecutá la celda siguiente y se descargará automáticamente el conjunto de entrenamiento y de validación.

In [ ]:
from torchvision.datasets import Imagenette
from torchvision.transforms import Compose, Grayscale, Resize, ToTensor

transform = Compose([Grayscale(), Resize((224, 224)), ToTensor()])
try:
  train_dataset = Imagenette(root='data', split="train", size="160px", transform=transform, download=False)
except RuntimeError:
  train_dataset = Imagenette(root='data', split="train", size="160px", transform=transform, download=True)
val_dataset = Imagenette(root='data', split="val", size="160px", transform=transform, download=False)

Para explorar el conjunto de validación, a continuación se muestra un *widget* interactivo que permite visualizar imágenes individuales junto con su etiqueta numérica y el nombre de la clase correspondiente. Moviendo el control deslizante se puede recorrer el dataset y observar ejemplos reales de las distintas categorías de *Imagenette*.

In [ ]:
@widgets.interact(idx=(0, len(val_dataset)-1))
def visualizar_imagen(idx):
    imagen, clase = val_dataset[idx]
    plt.imshow(imagen.squeeze(), cmap="gray", vmin=0, vmax=1)
    plt.show()
    print(f'Etiqueta: {clase}, Nombre: {val_dataset.classes[clase]}')

Mirá la escena que tenés enfrente: seguro podés distinguir sin problema los bordes que marcan la silueta del perrito, el contraste entre su pelo y el fondo, y hasta las sombras y detalles de su superficie. Esta facilidad para ver contornos refleja que en nuestro cerebro hay representaciones pensadas para procesar justamente esas características de borde.

En computación, una imagen no es más que una matriz de números donde cada valor indica la intensidad de un píxel. En la celda que sigue vas a poder ver la imagen completa del perrito junto con una región de interés marcada con un rectángulo azul. Esa región se puede mover usando el widget interactivo: simplemente desplazá los valores de x0 y y0 con los controles deslizantes para elegir qué parte de la imagen recortar y observar en detalle. A la derecha se muestra el recorte correspondiente, lo que permite explorar cómo se representa numéricamente cada sector de la imagen.

In [ ]:
import matplotlib.patches as patches
import ipywidgets as widgets

imagen, clase = val_dataset[600]
imagen = imagen.squeeze()
roi_size=20

@widgets.interact(x0=(0,imagen.shape[1]-roi_size), y0=(0,imagen.shape[0]-roi_size))
def roi(x0, y0):
  # Definimos una región de interés
  x1, y1 = x0 + roi_size, y0 + roi_size
  roi = imagen[y0:y1, x0:x1]

  # Graficamos ambas matrices con un rectángulo sobre la imagen original
  fig, (ax1, ax2) = plt.subplots(1, 2, layout="tight")
  
  ax1.imshow(imagen, cmap='gray')
  rect = patches.Rectangle((x0, y0), x1-x0, y1-y0,  linewidth=2, edgecolor='blue', facecolor='none')
  ax1.add_patch(rect)
  
  ax2.imshow(roi, cmap='gray')
  plt.show()

Probá con diferentes regiones de interés. Buscá una donde aparezca parte de la oreja del perrito. ¿El borde que se ve ahí es nítido?

Para tener una mejor intuición de los cambios de luminancia en la imagen, graficamos abajo los valores de luminancia a lo largo de una fila. También graficamos el cambio en los valores de luminancia, es decir, la diferencia entre pixeles consecutivos.

In [ ]:
@widgets.interact(x0=(0,imagen.shape[1]-roi_size), y=(0,imagen.shape[0]-1))
def roi(x0, y):
  # Definimos una región de interés
  x1 = x0 + roi_size
  roi = imagen[y, x0:x1]

  # Graficamos ambas matrices con un rectángulo sobre la imagen original
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
  
  ax1.imshow(imagen, cmap='gray')
  rect = patches.Rectangle((x0, y - 0.5), roi_size, 1, linewidth=2, edgecolor='blue', facecolor='none')
  ax1.add_patch(rect)
  
  ax2.set_title("Cambio de luminancia")
  ax2.plot(roi)
  ax2.set_ylim(-0.1, 1.1)
    
  plt.show()

Probá con diferentes filas y buscá un corte donde aparezcan las orejas del perrito. ¿Es fácil distinguir sus bordes a partir de las gráficas que muestran la diferencia de luminosidad?

## Convoluciones en una dimensión

Hasta acá exploramos la imagen del perrito y vimos cómo cambian las intensidades al mover una región de interés. Si ahora tomamos una sola fila que cruce, por ejemplo, el borde de la oreja, esas intensidades forman una señal en una dimensión: una lista de números que suben o bajan según el contraste entre el pelo y el fondo. Detectar ese borde en una dimensión se puede hacer con diferencias entre píxeles vecinos... o, más elegante y general, con una convolución usando un *kernel* chiquito. Primero lo vemos en 1D, y en un rato lo extendemos a 2D para operar sobre la imagen completa.

Para hacerlo revisaremos una operación matemática importante llamada **convolución**. Una convolución es una operación matemática que combina dos funciones para producir una tercera. En el contexto de procesamiento de imágenes y redes neuronales, la convolución es usada para extraer características importantes, como bordes, texturas y patrones.

Imaginá que tenés una imagen (que puedes ver como una matriz de píxeles) y un filtro o "kernel" (otra matriz más pequeña). La convolución consiste en deslizar este filtro sobre la imagen, multiplicando los valores de los píxeles por los valores del filtro y sumando los resultados en cada posición. Este proceso se repite a lo largo de toda la imagen, y el resultado es una nueva imagen donde se han resaltado ciertos rasgos de la imagen original, dependiendo del filtro usado.

Comenzaremos viendo que la convolución en una dimension. Luego la generalizaremos a dos dimensiones. Por ahora, el objetivo es detectar los bordes en una fila usando una convolución en lugar de la diferencia de píxeles consecutivos.

Supongamos que tenemos una señal $s$ dada por

$$s= \begin{bmatrix}0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 1\end{bmatrix}$$

Cuando realizamos una convolución de una señal con un filtro pequeño, como el filtro $f=[−1,1]$, el proceso consiste en superponer el filtro sobre los primeros elementos de la señal, multiplicar los elementos correspondientes y luego sumar los resultados para obtener el primer valor de la señal filtrada.

Por ejemplo, supongamos que estamos convolucionando la señal $s$ con el filtro $f$. La primera operación sería:

$$(f*s)(0)=f(0)*s(0)+f(1)*s(1)=(−1)*s(0)+(1)*s(1)=0$$

Luego, repetimos el cálculo desplazando el filtro una posición en la señal:

$$(f*s)(1)=f(0)*s(1)+f(1)*s(2)=(−1)*s(1)+(1)*s(2)=0$$

Hacemos esto para todas las posiciones posibles de la señal original. Al aplicar el filtro a toda la señal, obtenemos un nuevo vector, que en este caso es:

$$(f*s)=[0,0,0,0,1,0,0,0,0]$$

Este vector resultante representa la señal después de ser filtrada, donde las operaciones de convolución han resaltado un cambio en la señal en la posición central.

Veamos como hacerlo en Python usando la función de SciPy [`convolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve.html):

In [ ]:
s = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
f = np.array([-1, 1])
convolucion = -sp.signal.convolve(s, f, mode='valid')

print(f"s: {s}")
print(f"f: {f}")
print(f"Convolución: {convolucion}")

Para entender mejor cómo funciona la convolución en una dimensión, vamos a usar una función auxiliar que grafica tres cosas: la señal original, el filtro y el resultado de la convolución. De esta forma podemos ver de manera intuitiva cómo el filtro resalta los cambios en la señal.

In [ ]:
visualizar_convolucion1d(s, f, convolucion)

Podría ser interesante notar que la señal filtrada resultante, en este caso particular de filtro, está extrayendo justamente el cambio en la señal original.

Apliquémosla ahora al ejemplo del perrito.

In [ ]:
roi_size=20

out_img  = widgets.Output(layout=widgets.Layout())
out_conv = widgets.Output(layout=widgets.Layout())

@widgets.interact(x0=(0,imagen.shape[1]-roi_size), y=(0,imagen.shape[0]-1))
def roi(x0, y):
    # Definimos una región de interés
    x1 = x0 + roi_size
    roi = imagen[y, x0:x1]
    
    # Calculamos la convolución
    s = roi
    f = np.array([-1, 1])
    convolucion = sp.signal.convolve(s, np.flip(f), mode='valid')

    with out_img:
        out_img.clear_output(wait=True)
        fig, ax = plt.subplots(1, 1, figsize=(3, 3))
        ax.set_title("Región de interés")
        ax.imshow(imagen, cmap='gray')
        rect = patches.Rectangle((x0, y - 0.5), roi_size, 1, linewidth=2, edgecolor='blue', facecolor='none')
        ax.add_patch(rect)
        ax.set_axis_off()
        plt.show()
      
    with out_conv:
        out_conv.clear_output(wait=True)
        visualizar_convolucion1d(s, f, convolucion)

# Display the stacked boxes once
display(widgets.HBox([out_img, out_conv]))

¿Cómo se compara el resultado de aplicar esta convolución con el filtro $f=[-1, 1]$ respecto al cálculo directo de la diferencia de luminosidad entre dos píxeles consecutivos en la imagen del perrito? ¿Son iguales o presentan alguna diferencia?

El filtro $f=[1, 4, 7, 4, 1]$ corresponde a un ejemplo de filtro gaussiano suavizante. Probá realizar, en el mismo código anterior, una convolución con este filtro. ¿Qué efecto observás en la imagen del perrito? ¿Pensás que un suavizado de este tipo podría servir para resaltar bordes, o más bien cumple otra función?

## Convoluciones en dos dimensiones

Lo que vimos en 1D (tomar una fila y detectar cambios) ahora lo generalizamos a imágenes completas. En 2D, un filtro o kernel es una pequeña matriz (p. ej., $3 \times 3$, $5 \times 5$) que se desliza sobre la imagen: en cada posición multiplicamos elemento a elemento los píxeles del parche por los valores del kernel y sumamos. Ese valor va al píxel de salida.

Según el kernel, podemos suavizar (gaussiano), realzar bordes (Sobel/Prewitt/Laplaciano) o resaltar texturas.

En la celda que sigue definimos un kernel 2D y lo aplicamos a la imagen del perrito. Tips útiles:
* Usá tamaños impares (3, 5, 7, etc) para que el kernel tenga un centro bien definido.
* Los kernels de borde suelen sumar 0 (responden a cambios, no a regiones planas).
* Los de suavizado conviene normalizarlos para conservar el brillo promedio.

Usaremos un kernel tipo Prewitt en x (bordes verticales). Para una visualización más clara, miraremos el valor absoluto de la respuesta (o la magnitud del gradiente).

In [ ]:
s = imagen
f = np.array([
    [-1, 0, 1],
    [-1, 0, 1],
    [-1, 0, 1]
])
convolucion = sp.signal.convolve(s, np.flip(f), mode='same')
visualizar_convolucion2d(s, f, convolucion)

Ahora probamos el mismo detector en x pero con polaridad invertida, usando el kernel

$$\begin{bmatrix}
1 & 0 & -1\\
1 & 0 & -1\\
1 & 0 & -1
\end{bmatrix}$$

Debería resaltar los mismos bordes que antes, pero con signo opuesto (oscuro a claro vs claro a oscuro).

In [ ]:
f = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
])
convolucion = sp.signal.convolve(s, np.flip(f), mode='same')
visualizar_convolucion2d(s, f, convolucion)

Otros ejemplos de filtros de convolución son:

$$sharpen = \begin{bmatrix}0 & -1 & 0 \\ -1 & 5 & -1 \\ 0 & 1 & 0\end{bmatrix}$$

$$emboss = \begin{bmatrix}-2 & -1 & 0 \\ -1 & 1 & 1 \\ 0 & 1 & 2\end{bmatrix}$$

$$outline = \begin{bmatrix}-1 & -1 & -1 \\ -1 & 8 & -1 \\ -1 & -1 & -1\end{bmatrix}$$

$$top sobel = \begin{bmatrix}1 & 2 & 1 \\ 0 & 0 & 0 \\ -1 & -2 & -1\end{bmatrix}$$

$$right sobel = \begin{bmatrix}-1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1\end{bmatrix}$$

## Bonificación 1: Filtros de detección de contraste

En el procesamiento temprano de la información visual, un mecanismo fundamental consiste en resaltar las diferencias de luminancia entre regiones vecinas de la imagen. Para ello se utilizan filtros de detección de contraste, diseñados con una organización de tipo centro-periferia: el píxel central ejerce una influencia excitatoria, mientras que el entorno inmediato contribuye de manera inhibitoria. Al equilibrar estas dos zonas de modo que los pesos totales sumen cero, el filtro se vuelve sensible únicamente a los cambios locales de intensidad, suprimiendo áreas uniformes y potenciando los bordes y transiciones que marcan la estructura de la escena. Este tipo de operación no es solo un recurso computacional: tiene un correlato biológico en la organización de los campos receptivos de las células ganglionares de la retina y de las neuronas en el núcleo geniculado lateral, que responden de manera selectiva a contrastes espaciales semejantes.

In [ ]:
f = np.array([
    [1/8, 1/8, 1/8],
    [1/8, -1, 1/8],
    [1/8, 1/8, 1/8]
])
convolucion = sp.signal.convolve(s, np.flip(f), mode='same')
visualizar_convolucion2d(s, f, convolucion)

In [ ]:
f = np.array([
    [-1/8, -1/8, -1/8],
    [-1/8,  1,   -1/8],
    [-1/8, -1/8, -1/8]
])
convolucion = sp.signal.convolve(s, np.flip(f), mode='same')
visualizar_convolucion2d(s, f, convolucion)

Anteriormente vimos el ejemplo de aplicar un filtro gaussiano en una dimensión. Ahora veremos que sucede si, antes de aplicar un filtro de detección de bordes, aplicamos un filtro gausiano:

In [ ]:
f1 = np.array([
    [1/16, 2/16, 1/16],
    [2/16, 4/16, 2/16],
    [1/16, 2/16, 1/16]
])
f2 = np.array([
    [-1/8, -1/8, -1/8],
    [-1/8, 1, -1/8],
    [-1/8, -1/8, -1/8]
])

primera_convolucion = sp.signal.convolve(s, np.flip(f1))
segunda_convolucion = sp.signal.convolve(primera_convolucion, np.flip(f2))

visualizar_convolucion2d(s, f1, primera_convolucion)
visualizar_convolucion2d(primera_convolucion, f2, segunda_convolucion)

¿Mejora la detección de bordes? ¿Por qué?

## Bonificación 2: nuestra segunda red convolucional

En esta sección volvemos a las redes neuronales convolucionales (CNN) para conectarlas con lo que ya hicimos (diferencias, convoluciones 1D/2D y detección de bordes). Las CNN capturan, de forma simple, varias ideas que sabemos del sistema visual biológico:

1. Una **jerarquía** de procesamiento (de bordes locales a formas y objetos), análoga al recorrido V1 → V2 → V4 → IT
2. **Campos receptivos** que crecen con la profundidad, como pasa en corteza visual

En investigación, estas propiedades facilitan construir **modelos de codificación** de respuestas neuronales, comparar **representaciones internas** del modelo con actividad cerebral (p. ej., con RSA) y testear hipótesis sobre cómo se **extrae información visual** que luego guía la conducta.

**¿Qué vamos a construir?**  

Una CNN “en etapas” (inspirada en el flujo V1 → V2 → IT) con bloques `Conv2d → BatchNorm2d → ReLU`, *pooling* en las primeras etapas y un cierre con `AdaptiveAvgPool2d(1) → Linear`. Esta estructura:

- **Estabiliza** el entrenamiento con `BatchNorm2d` (aprende más rápido y con LR razonable).
- Es **agnóstica al tamaño** de entrada gracias a `AdaptiveAvgPool2d(1)` (no hay que calcular dimensiones para la capa final).
- Mantiene la **intuición** de los filtros: las primeras capas detectan cambios locales (bordes, texturas); las siguientes combinan esas pistas.

**Qué vamos a observar**

1. **Baseline simple**: entrenar la red tal cual (inicialización por defecto) y medir la accuracy de validación.
2. **Visualización**: inspeccionar los **mapas de activación** de la primera etapa sobre la imagen del perrito para ver qué regiones resaltan (por ejemplo, contornos de orejas y lomo).

**Objetivo**

Reforzar el puente entre los **kernels** trabajados a mano y una **CNN entrenable** que aprende a combinarlos para reconocer clases.

El bloque de abajo implementa una red inspirada en **CORnet-Z**: organiza el flujo en etapas análogas a **V1 → V2 → IT**, donde cada etapa aplica `Conv2d → BatchNorm2d → ReLU` y las primeras dos incluyen *pooling* para aumentar el campo receptivo y ganar invariancia a traslación.

- **V1 y V2**: detectan patrones locales (bordes/texturas) y reducen resolución con `MaxPool2d`.  
- **IT**: combina rasgos de mayor escala con otra conv + BN (sin *pooling*) para preservar detalle.  
- **Cabeza**: `AdaptiveAvgPool2d(1)` comprime cualquier tamaño a un vector por canal (**agnóstico al tamaño de entrada**) y `Linear(128, 10)` produce las clases.

En el `forward` se guardan activaciones intermedias como `V1`, `V2` e `IT`, útiles para **visualizar mapas de activación** y discutir qué “ve” cada área del modelo (tal como se hace al analizar representaciones en neurociencia computacional).

In [ ]:
class SimpleCNN(nn.Module):
      
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1, bias=False)
    self.bn1   = nn.BatchNorm2d(64)
    self.pool1 = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1, bias=False)
    self.bn2   = nn.BatchNorm2d(128)
    self.pool2 = nn.MaxPool2d(2, 2)
    self.conv3 = nn.Conv2d(128, 128, 3, padding=1, bias=False)
    self.bn3   = nn.BatchNorm2d(128)
    self.avg = nn.AdaptiveAvgPool2d(1) 
    self.fc   = nn.Linear(128, 10)  
    
  def forward(self, x):
    V1 = F.relu(self.bn1(self.conv1(x)))
    V1 = self.pool1(V1)
    V2 = F.relu(self.bn2(self.conv2(V1)))
    V2 = self.pool2(V2)
    it = F.relu(self.bn3(self.conv3(V2)))
    x = self.avg(it).flatten(1)
    x = self.fc(x)
    return x

En el bloque que sigue entrenamos nuestra red sobre *Imagenette* y medimos su desempeño en validación. Definimos los *dataloaders* de train/val, elegimos **Adam** como optimizador y **CrossEntropyLoss** como función de pérdida, y fijamos semillas para mantener la **reproducibilidad**. El ciclo recorre varias **épocas**; en cada una entrenamos con `train_net(...)`, evaluamos con `eval_net(...)` y reportamos la **precisión** en validación.

El objetivo es ver cómo evoluciona la *accuracy* a medida que la red aprende y, si hace falta, ajustar hiperparámetros (tamaño de batch, tasa de aprendizaje, número de épocas). Probá aumentando el número de épocas a 5.

In [ ]:
from torch.utils.data import DataLoader

device = encontrar_device()

cnn = SimpleCNN().to(device)

# Achicamos los datasets para que sean mas rápido de entrenar
transform = Compose([Grayscale(), Resize((64, 64)), ToTensor()])
train_dataset = Imagenette(root='data', split="train", size="160px", transform=transform, download=False)
val_dataset = Imagenette(root='data', split="val", size="160px", transform=transform, download=False)

# Cargamos los datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Especificamos el optimizador
optimizer_cnn = torch.optim.Adam(cnn.parameters(), lr=1e-3)

# Especificamos el criterio para medir la pérdida
criterion = nn.CrossEntropyLoss()

# Reproducibilidad
torch.manual_seed(123)
np.random.seed(123)

# Iteramos sobre una cierta cantidad de épocas
tmax = 1
for ep in range(1, tmax+1):
  # Entrenamos a la red en el conjunto de entrenamiento
  tr_loss, tr_acc = train_net(cnn, train_loader, optimizer_cnn, criterion)

  # Evaluamos en el conjunto de evaluación
  va_loss, va_acc = eval_net(cnn, val_loader, criterion)

  print(f"Época {ep:02d} | Precisión={va_acc:.4f}")

### Visualizar los filtros aprendidos (primera capa)

Para cerrar, miramos qué aprendió la primera capa. El comando de abajo dibuja los 64 kernels de `conv1` en una grilla. Fijate si aparecen patrones de borde (horizontales, verticales, diagonales), simetrías o texturas locales: eso conecta directamente con la intuición de las convoluciones 2D que trabajamos antes.

In [ ]:
visualizar_pesos(cnn.conv1, in_channel=0)